In [3]:
# Parameters: 
E = [40]   # (kPa)
H = [20]      # (mm)
T = [ 4 ]  #[1, 2, 3, 4]      # (mm)

# Angles_x = [ 0 ] # (Deg)
# Angles_Z = [ 0 ]  # (Deg)
Angles_x = list(range(0,13,2)) # (Deg)
Angles_Z = list(range(0,61,5)) # (Deg)

NoCPUs = 10

Complete_prev_simulations = True   # (yes/ no)

N_list = [
        # [1, 6],
        # [1, 6, 12 , 18],
        # [1, 6, 12 , 18, 24, 30],
        [1, 6, 12 , 18, 24, 30, 36, 42],
        ]
Dia_list = [
        # [0.0, 18.0],
        # [0.0, 10.0, 20.0, 30.0],
        # [0.0, 6.0, 12.0, 18.0, 24.0, 30.0],
        [0.0, 5.0, 10.0, 15.0, 20.0, 25.0 ,30.0, 35.0],
        ]

if len(N_list) != len(Dia_list):
    print('Error: N_list and Dia_list must have the same length')
    exit()


# sum the elments in each list in N_list
E = [sum(i) for i in N_list]

NoCases = len(E)*len(H)*len(T)*len(Angles_x)*len(Angles_Z)
NoCases


91

In [4]:
# Simualtion code 

import os
import numpy as np
import subprocess
import shutil
from time import sleep, time
from tqdm import tqdm

start_time = time()
def run_abaqus(e,h,t,NoCPUs,angle_x,angle_z,period,N_list,Dia_list,depth, meshSize):
    abaqusCode = abaqusCode_main
    python_file_path = os.path.join(Thickness_path_current,"Case.py")
    cae_file_path = os.path.join(Thickness_path_current,"Case.cae").replace(os.path.sep, chr(92))  # chr(92) = \
    parameters = {"!@0000E":str(float(e)),
              "!@0000H":str(float(h)),
              "!@0000T":str(float(t)),
              "!@0000CAE":str(cae_file_path),
              "!@0000CPUs":str(NoCPUs),
              "!@0000Angle_X":str(float(angle_x)),
              "!@0000Angle_Z":str(float(angle_z)),
              "!@0000Period":str(float(period)),
              "!@0000N_list":str(N_list),
              "!@0000Dia_list":str(Dia_list),
              "!@0000Depth":str(depth),
              "!@0000ML":str(meshSize)
              } 

    for p in list(parameters.keys()): 
        abaqusCode = abaqusCode.replace(p,parameters[p])   # operate by replacing the suitable values in .py file 
    
    file = open(python_file_path, 'w')
    file.write(abaqusCode)
    file.close()
    # subprocess.Popen("abaqus cae noGUI=Case.py", cwd=Thickness_path_current, shell=True, stdout=subprocess.PIPE)  ## script / noGUI
    subprocess.Popen("abaqus cae noGUI=Case.py", cwd=Thickness_path_current, shell=True, stdout=subprocess.PIPE)  ## script / noGUI

    print(''.join(str(Thickness_path_current).split('\\')[-5:])," Case has been submitted")    
    sleep(5)

def two_cases_parallel():
    x = 1


home_dir = os.path.join(os.getcwd() ,"Parametric Study")
try:
    os.mkdir(home_dir)
except:
    pass

abaqusCode_main = open('ABAQUS_PySc.py','r').read()
cae_trigger_path_previous = ""
Thickness_path_previous = ""
cae_trigger_path_pre_previous = ""
Thickness_path_pre_previous = ""


for i in E: 
    i
    material_path = os.path.join(home_dir,"Dist=%smarkers" % i)
    try:      #### Create the material subfolders if not exist
        os.mkdir(material_path)
    except:
        pass
    
    N = N_list[E.index(i)]
    Dia = Dia_list[E.index(i)]
    for j in H:
        Height_path = os.path.join(material_path,"H=%.2fmm" % j)
        try:      #### Create the height subfolders if not exist
            os.mkdir(Height_path)
        except:
            pass
        
        for angle_x in Angles_x:
            angle_X_path = os.path.join(Height_path,"Ang_X=%.2f" % angle_x)
            try: 
                os.mkdir(angle_X_path)
            except: 
                pass

            for angle_z in Angles_Z:
                angle_Z_path = os.path.join(angle_X_path,"Ang_Z=%.2f" % angle_z)
                try:
                    os.mkdir(angle_Z_path)
                except:
                    pass

                for k in T:
                    Thickness_path_current = os.path.join(angle_Z_path,"T=%.2fmm" % k)
                    cae_trigger_path_current = os.path.join(Thickness_path_current,"NextCaseTrigger-CaseDone.txt").replace(os.path.sep, chr(92))  # chr(92) = \
                    
                    try:    
                        os.mkdir(Thickness_path_current)
                        run_abaqus(i, j, k, NoCPUs, angle_x, angle_z, 0.1,    N,      Dia,      0.01,   0.001 )
                        #run_abaqus(e,h,t,  NoCPUs, angle_x, angle_z, period, N_list, Dia_list, depth, meshSize)
                        sleep(5)
                    except:
                        pass

                    while os.path.exists(cae_trigger_path_previous) == False and cae_trigger_path_previous != '':
                        sleep(5)
                    
                    
                    Thickness_path_previous = Thickness_path_pre_previous
                    cae_trigger_path_previous = cae_trigger_path_pre_previous
                    Thickness_path_previous = Thickness_path_current
                    cae_trigger_path_previous = cae_trigger_path_current

while True:
    if os.path.exists(cae_trigger_path_current)==True:
        break
    sleep(5)

end_time = time()

processing_time = end_time-start_time
mean_processing_time = processing_time / NoCases
print('Summary: \nNumber of cases = ',NoCases, ' (cases)\ntotla processing time = ',processing_time/60,' (min)\nMean processing time = ',mean_processing_time/60, '(min/case)')

Dist=169markersH=20.00mmAng_X=2.00Ang_Z=45.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=2.00Ang_Z=50.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=2.00Ang_Z=55.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=2.00Ang_Z=60.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=0.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=5.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=10.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=15.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=20.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=25.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=30.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=35.00T=4.00mm  Case has been submitted
Dist=169markersH=20.00mmAng_X=4.00Ang_Z=40.00T=4.00mm 

In [7]:
import os
print (os.cpu_count ()) 

24
